In [1]:
# Install Unsloth
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-n6ufkv_m/unsloth_e3d776a79c4f488297195146e84b6d8a
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-n6ufkv_m/unsloth_e3d776a79c4f488297195146e84b6d8a
  Resolved https://github.com/unslothai/unsloth.git to commit 1c0ad844f170f67c7cdf6f7a9465bafb0f9627df
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 100.7 MB/s eta 0:00:00

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

# Load SmolLM2-135M model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/SmolLM2-135M-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("✅ Model loaded successfully!")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/423 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

✅ Model loaded successfully!


In [3]:
# Configure STANDARD LoRA (r=16) - Parameter Efficient!
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # Standard LoRA rank (vs r=128 in Colab 1)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # Typically same as r
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("✅ Standard LoRA (r=16) configured!")
model.print_trainable_parameters()

Unsloth 2025.11.2 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


✅ Standard LoRA (r=16) configured!
trainable params: 4,884,480 || all params: 139,400,064 || trainable%: 3.5039


In [4]:
from datasets import load_dataset

# Load the SAME Python coding dataset for comparison
dataset = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train")

print(f"✅ Dataset loaded: {len(dataset)} examples")
print("\nExample:")
print(dataset[0])

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001-8b6e212f3e1ece(…):   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

✅ Dataset loaded: 18612 examples

Example:
{'instruction': 'Create a function to calculate the sum of a sequence of integers.', 'input': '[1, 2, 3, 4, 5]', 'output': '# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum', 'prompt': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a function to calculate the sum of a sequence of integers.\n\n### Input:\n[1, 2, 3, 4, 5]\n\n### Output:\n# Python code\ndef sum_sequence(sequence):\n  sum = 0\n  for num in sequence:\n    sum += num\n  return sum'}


In [5]:
# Alpaca prompt template
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched=True)
print("✅ Dataset formatted!")

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

✅ Dataset formatted!


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,  # Same as Colab 1 for fair comparison
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",  # Disable wandb
    ),
)

print("✅ Trainer configured!")

Unsloth: We found double BOS tokens - we shall remove one automatically.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/18612 [00:00<?, ? examples/s]

✅ Trainer configured!


In [7]:
# Start training
import time
start_time = time.time()

trainer_stats = trainer.train()

training_time = time.time() - start_time
print(f"\n⏱️ Training completed in {training_time/60:.2f} minutes")

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 18,612 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.948900
2,1.823200
3,1.620300
4,1.671900
5,1.586800
6,1.680100
7,1.528700
8,1.600900
9,1.837100
10,1.513300



⏱️ Training completed in 2.15 minutes


In [8]:
# Enable fast inference
FastLanguageModel.for_inference(model)

# Test with same prompt as Colab 1 for comparison
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Write a Python function to calculate the factorial of a number.",
        "",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
print("📝 Generated Code:")
print(tokenizer.batch_decode(outputs)[0])

📝 Generated Code:
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Write a Python function to calculate the factorial of a number.

### Input:


### Response:
import math

def factorial(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    elif n == 0 or n == 1:
        return 1
    else:
        result = 1
        for i in range(1, n):
            result *= i
        return result

# Example usage:
print(factorial(5))

# Output:
# 5

### Response:
import math

def factorial(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative


In [9]:
# Test multiple coding tasks
test_prompts = [
    "Write a Python function to reverse a string.",
    "Create a function to check if a number is prime.",
    "Write a function to find the largest element in a list.",
]

for prompt in test_prompts:
    print(f"\n{'='*60}")
    print(f"📝 Instruction: {prompt}")
    print('='*60)

    inputs = tokenizer(
        [alpaca_prompt.format(prompt, "", "")],
        return_tensors = "pt"
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
    response = tokenizer.batch_decode(outputs)[0]

    # Extract only the response part
    response_start = response.find("### Response:") + len("### Response:")
    clean_response = response[response_start:].strip()
    print(clean_response)


📝 Instruction: Write a Python function to reverse a string.
def reverse_string(input_str):
    return input_str[::-1]<|im_end|>

📝 Instruction: Create a function to check if a number is prime.
import math

def is_prime(n):
    if n <= 0:
        return False
    for i in range(2, int(math.sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

# Example usage:
print(is_prime(10))

# Output:
# False

# Explanation:
# 10 is not prime because it has a divisor of 2 that is not divisible by any number that is greater than 1.
# 9 is prime because it has a divisor

📝 Instruction: Write a function to find the largest element in a list.
def find_largest(lst):
    # Write code here

# Example usage:
largest_element = find_largest([1, 2, 3, 4, 5])
print(largest_element)<|im_end|>


In [10]:
# Save LoRA adapters
save_dir = "smollm2_python_lora_r16"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"✅ Model saved to: {save_dir}")

# Check file sizes
!du -sh {save_dir}
!ls -lh {save_dir}

✅ Model saved to: smollm2_python_lora_r16
24M	smollm2_python_lora_r16
total 24M
-rw-r--r-- 1 root root 1.1K Nov  9 20:25 adapter_config.json
-rw-r--r-- 1 root root  19M Nov  9 20:25 adapter_model.safetensors
-rw-r--r-- 1 root root   29 Nov  9 20:25 added_tokens.json
-rw-r--r-- 1 root root  368 Nov  9 20:25 chat_template.jinja
-rw-r--r-- 1 root root 456K Nov  9 20:25 merges.txt
-rw-r--r-- 1 root root 5.2K Nov  9 20:25 README.md
-rw-r--r-- 1 root root  537 Nov  9 20:25 special_tokens_map.json
-rw-r--r-- 1 root root 3.7K Nov  9 20:25 tokenizer_config.json
-rw-r--r-- 1 root root 3.4M Nov  9 20:25 tokenizer.json
-rw-r--r-- 1 root root 782K Nov  9 20:25 vocab.json
